In [ ]:
import librosa
import os
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
DATASET_PATH = '/content/drive/MyDrive/Datasetyolanda'

In [ ]:
JSON_PATH = "/content/drive/MyDrive/tardigradeaudio.json"

In [ ]:
SAMPLES_TO_CONSIDER = 22050

In [ ]:
def prepare_dataset(dataset_path, json_path, n_mfcc=13, hop_length=512, n_fft=2048):
    data = {
        "mappings": [],
        "labels": [],
        "MFCCs": [],
        "files": []
    }

    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath is not dataset_path:
            category = dirpath.split("/")[-1]
            data["mappings"].append(category)
            print(f"Processing{category}")

            for f in filenames:
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path)

                if len(signal) >= SAMPLES_TO_CONSIDER:
                    signal = signal[:SAMPLES_TO_CONSIDER]
                    MFCCs = librosa.feature.mfcc(signal, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)

                    data["labels"].append(i - 1)
                    data["MFCCs"].append(MFCCs.T.tolist())
                    data["files"].append(file_path)
                    print(f"{file_path}: {i - 1}")

    with open(JSON_PATH, "w") as fp:
        json.dump(data, fp, indent=4)


if __name__ == "__main__":
    prepare_dataset(DATASET_PATH, JSON_PATH)

In [ ]:
import numpy as np
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

In [ ]:
DATA_PATH = "/content/drive/MyDrive/tardigradeaudio.json"
SAVED_MODEL_PATH = "/content/drive/MyDrive/tardigradeaudiomodel.h5"

In [ ]:
LEARNING_RATE = 0.0001
EPOCHS = 1000
BATCH_SIZE = 32

In [ ]:
NUM_KEYWORDS = 10

In [ ]:
def load_dataset(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["MFCCs"])
    y = np.array(data["labels"])

    return X,y

In [ ]:
def get_data_splits(data_path, test_size=0.1, test_validation=0.1):
    X, y = load_dataset(data_path)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=test_validation)

    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test


In [ ]:
def build_model(input_shape, learning_rate, error="sparse_categorical_crossentropy"):

    model = keras.Sequential()

    #layer1

    model.add(keras.layers.Conv2D(64,(3,3), activation="relu",
                                  input_shape=input_shape,
                                  kernel_regularizer=keras.regularizers.l2(0.001)))

    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2,2),padding="same"))

    # layer2
    model.add(keras.layers.Conv2D(32, (3, 3), activation="relu",
                                  kernel_regularizer=keras.regularizers.l2(0.001)))

    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding="same"))

    #layer 3
    model.add(keras.layers.Conv2D(32, (2, 2), activation="relu",
                                  kernel_regularizer=keras.regularizers.l2(0.001)))

    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPool2D((2, 2), strides=(2, 2), padding="same"))

    #flatten
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation = "relu"))
    model.add(keras.layers.Dropout(0.3))

    #softmax
    model.add(keras.layers.Dense(NUM_KEYWORDS, activation="softmax"))

    #compile
    optimiser = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimiser, loss=error, metrics=["accuracy"])

    #print model
    model.summary()

    return model


In [ ]:
def main():
    X_train, X_validation, X_test, y_train, y_validation, y_test = get_data_splits(DATA_PATH)

    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    model = build_model(input_shape, LEARNING_RATE)

    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,
              validation_data = (X_validation, y_validation))

    test_error, test_accuracy = model.evaluate(X_test, y_test)
    print(f"Test error: {test_error}, test accuracy: {test_accuracy}")

    model.save(SAVED_MODEL_PATH)

In [ ]:
if __name__ == "__main__":
    main()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 42, 11, 64)        640       
                                                                 
 batch_normalization_3 (Batc  (None, 42, 11, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 21, 6, 64)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 19, 4, 32)         18464     
                                                                 
 batch_normalization_4 (Batc  (None, 19, 4, 32)        128       
 hNormalization)                                                 
                                                      